In [1]:
!wget https://raw.githubusercontent.com/discourse-lab/en_dimlex/master/en_dimlex.xml

--2023-08-17 14:26:23--  https://raw.githubusercontent.com/discourse-lab/en_dimlex/master/en_dimlex.xml
Auflösen des Hostnamens raw.githubusercontent.com (raw.githubusercontent.com) … 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Verbindungsaufbau zu raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443 … verbunden.
HTTP-Anforderung gesendet, auf Antwort wird gewartet … 200 OK
Länge: 208789 (204K) [text/plain]
Wird in ‘en_dimlex.xml.1’ gespeichert.

en_dimlex.xml.1     100%[===================>] 203,90K  --.-KB/s    in 0,02s   

2023-08-17 14:26:23 (8,22 MB/s) - ‘en_dimlex.xml.1’ gespeichert [208789/208789]



In [1]:
import xml.etree.ElementTree as ET
from utils import connective_string
from collections import Counter

import json
import pandas as pd

/home/HDD2TB/conda_envs/argmining-pattern/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
claim_connective_premise = ["conversely", "whereas", "but", "however", "while", "although", "though", "meanwhile", "previously", "instead", "or", "and", "then", "when", "as", "for", "accordingly", "because", "with", "since", "as", "and", "when", "if", "but"]
premise_connective_claim = ["yet", "still", "nevertheless", "rather", "nonetheless", "therefore", "thereby", "so", "hence", "consequently", "thus"]

In [2]:

tree = ET.parse('en_dimlex.xml')
root = tree.getroot()

In [3]:
entries = root.findall("entry")
len(entries)

142

In [40]:
freq_dict = dict()
for entry in entries:
    i = 0
    supporting_freq = 0
    attacking_freq = 0
    word = entry.get("word")
    ptb_relations = list()
    if not " " in word:
        # for syn in entry.findall(".//syn"):
        #     ptb_relations.extend(syn.findall(".//pdtb2_relation"))
        ptb_relations = entry.findall(".//pdtb2_relation")
        for rel in ptb_relations:
            freq = rel.get("pdtb_freq")
            sense = rel.get("sense")
            if freq:
                i += int(freq)
                if sense.lower().startswith("comparison.contrast"):
                    attacking_freq += int(freq)
                elif sense.lower().startswith("contingency.cause.result") or sense.lower().startswith("contingency.cause.reason"):
                    supporting_freq += int(freq)
            else:
                i += 1
                if sense.lower().startswith("comparison.contrast"):
                    attacking_freq += 1
                elif sense.lower().startswith("contingency.cause.result") or sense.lower().startswith("contingency.cause.reason"):
                    supporting_freq += 1
        
        if word == "if":
            print(entry, len(ptb_relations), attacking_freq, supporting_freq)
        freq_dict[word] = {"freq": i, "senses": len(set(ptb_relations)), "attacking": attacking_freq, "supportive": supporting_freq,
                          "attacking_perc": round((attacking_freq/i)*100,2), "supportive_perc": round((supporting_freq/i)*100,2)}

# print(len(freq_dict), dict(sorted(freq_dict.items(), key=lambda item: item[1], reverse=True)))
freq_dict


<Element 'entry' at 0x7f9929cdf400> 25 5 1


{'once': {'freq': 84,
  'senses': 7,
  'attacking': 0,
  'supportive': 0,
  'attacking_perc': 0.0,
  'supportive_perc': 0.0},
 'although': {'freq': 328,
  'senses': 8,
  'attacking': 157,
  'supportive': 0,
  'attacking_perc': 47.87,
  'supportive_perc': 0.0},
 'though': {'freq': 320,
  'senses': 23,
  'attacking': 152,
  'supportive': 0,
  'attacking_perc': 47.5,
  'supportive_perc': 0.0},
 'but': {'freq': 3308,
  'senses': 18,
  'attacking': 2424,
  'supportive': 1,
  'attacking_perc': 73.28,
  'supportive_perc': 0.03},
 'because': {'freq': 858,
  'senses': 5,
  'attacking': 0,
  'supportive': 854,
  'attacking_perc': 0.0,
  'supportive_perc': 99.53},
 'nevertheless': {'freq': 44,
  'senses': 6,
  'attacking': 19,
  'supportive': 0,
  'attacking_perc': 43.18,
  'supportive_perc': 0.0},
 'before': {'freq': 326,
  'senses': 5,
  'attacking': 0,
  'supportive': 0,
  'attacking_perc': 0.0,
  'supportive_perc': 0.0},
 'until': {'freq': 162,
  'senses': 8,
  'attacking': 0,
  'supportive':

In [4]:
freq_dict = dict()
for entry in entries:
    i = 0
    supporting_freq = 0
    attacking_freq = 0
    word = entry.get("word")
    if not " " in word:
        ptb_relations = entry.findall(".//pdtb2_relation")
        for rel in ptb_relations:
            freq = rel.get("pdtb_freq")
            sense = rel.get("sense")
            if freq:
                i += int(freq)
                if sense.lower().startswith("comparison.contrast"):
                    attacking_freq += int(freq)
                elif sense.lower().startswith("contingency.cause.result") or sense.lower().startswith("contingency.cause.reason"):
                    supporting_freq += int(freq)
            else:
                i += 1
                if sense.lower().startswith("comparison.contrast"):
                    attacking_freq += 1
                elif sense.lower().startswith("contingency.cause.result") or sense.lower().startswith("contingency.cause.reason"):
                    supporting_freq += 1
        freq_dict[word] = {"freq": i, "senses": len(set(ptb_relations)), "attacking": attacking_freq, "supportive": supporting_freq,
                          "attacking_perc": round((attacking_freq/i)*100,2), "supportive_perc": round((supporting_freq/i)*100,2)}

# print(len(freq_dict), dict(sorted(freq_dict.items(), key=lambda item: item[1], reverse=True)))
freq_dict


{'once': {'freq': 84,
  'senses': 7,
  'attacking': 0,
  'supportive': 0,
  'attacking_perc': 0.0,
  'supportive_perc': 0.0},
 'although': {'freq': 328,
  'senses': 8,
  'attacking': 157,
  'supportive': 0,
  'attacking_perc': 47.87,
  'supportive_perc': 0.0},
 'though': {'freq': 320,
  'senses': 23,
  'attacking': 152,
  'supportive': 0,
  'attacking_perc': 47.5,
  'supportive_perc': 0.0},
 'but': {'freq': 3308,
  'senses': 18,
  'attacking': 2424,
  'supportive': 1,
  'attacking_perc': 73.28,
  'supportive_perc': 0.03},
 'because': {'freq': 858,
  'senses': 5,
  'attacking': 0,
  'supportive': 854,
  'attacking_perc': 0.0,
  'supportive_perc': 99.53},
 'nevertheless': {'freq': 44,
  'senses': 6,
  'attacking': 19,
  'supportive': 0,
  'attacking_perc': 43.18,
  'supportive_perc': 0.0},
 'before': {'freq': 326,
  'senses': 5,
  'attacking': 0,
  'supportive': 0,
  'attacking_perc': 0.0,
  'supportive_perc': 0.0},
 'until': {'freq': 162,
  'senses': 8,
  'attacking': 0,
  'supportive':

In [44]:
def filter_connectives(dict_connectives, rel, threshold, add_perc=False, add_senses=False):
    list_conn = list()
    for conn, conn_values in dict_connectives.items():
        if type(rel) == str:
            if conn_values[rel] >= threshold:
                if add_perc and add_senses:
                    list_conn.append([conn, conn_values[rel], conn_values["freq"]])
                else:
                    list_conn.append(conn)
        elif type(rel) == list:
            if conn_values[rel[0]] >= threshold and conn_values[rel[1]] >= threshold:
                if add_perc and add_senses:
                    list_conn.append([conn, conn_values[rel], conn_values["freq"]])
                else:
                    list_conn.append(conn)
    return list_conn

In [45]:
conn_attack_all = filter_connectives(freq_dict, "attacking_perc", 0.01)
conn_attack_all 

['although',
 'though',
 'but',
 'nevertheless',
 'if',
 'previously',
 'when',
 'and',
 'then',
 'while',
 'however',
 'still',
 'or',
 'instead',
 'as',
 'nonetheless',
 'meanwhile',
 'yet',
 'rather',
 'besides',
 'nor',
 'whereas',
 'earlier',
 'conversely']

In [46]:
conn_support_all = filter_connectives(freq_dict, "supportive_perc", 0.01)
conn_support_all

['but',
 'because',
 'if',
 'when',
 'and',
 'so',
 'then',
 'as',
 'since',
 'therefore',
 'thus',
 'thereby',
 'consequently',
 'hence',
 'accordingly',
 'for',
 'with',
 'given']

In [47]:
conn_attack_20 = filter_connectives(freq_dict, "attacking_perc", 20)
conn_attack_20

['although',
 'though',
 'but',
 'nevertheless',
 'while',
 'however',
 'still',
 'yet',
 'rather',
 'whereas',
 'conversely']

In [48]:
conn_support_20 = filter_connectives(freq_dict, "supportive_perc", 20)
conn_support_20 

['because',
 'so',
 'as',
 'since',
 'therefore',
 'thus',
 'thereby',
 'consequently',
 'hence',
 'accordingly',
 'for',
 'with',
 'given']

In [49]:
conn_attack_40 = filter_connectives(freq_dict, "attacking_perc", 40)
conn_attack_40 

['although',
 'though',
 'but',
 'nevertheless',
 'while',
 'however',
 'still',
 'yet',
 'whereas',
 'conversely']

In [50]:
conn_support_40 = filter_connectives(freq_dict, "supportive_perc", 40)
conn_support_40 

['because',
 'so',
 'since',
 'therefore',
 'thus',
 'thereby',
 'consequently',
 'hence',
 'accordingly',
 'for',
 'with']

In [51]:
conn_attack_80  = filter_connectives(freq_dict, "attacking_perc", 60)
conn_attack_80 

['but', 'however', 'whereas', 'conversely']

In [52]:
conn_support_80 = filter_connectives(freq_dict, "supportive_perc", 80)
conn_support_80 

['because',
 'so',
 'therefore',
 'thus',
 'thereby',
 'consequently',
 'hence',
 'accordingly',
 'for']

In [53]:
conn_attack_support_all = filter_connectives(freq_dict, ["attacking_perc", "supportive_perc"], 0.01)
conn_attack_support_all 

['but', 'if', 'when', 'and', 'then', 'as']

In [54]:
set(freq_dict.keys()).difference(set(conn_attack_all)).difference(set(conn_support_all))

{'additionally',
 'after',
 'afterward',
 'also',
 'alternatively',
 'anyway',
 'before',
 'despite',
 'else',
 'essentially',
 'eventually',
 'everytime',
 'except',
 'finally',
 'further',
 'furthermore',
 'indeed',
 'later',
 'lest',
 'likewise',
 'meantime',
 'moreover',
 'next',
 'once',
 'otherwise',
 'overall',
 'particularly',
 'plus',
 'regardless',
 'separately',
 'similarly',
 'simultaneously',
 'specifically',
 'thereafter',
 'till',
 'ultimately',
 'unless',
 'until',
 'upon',
 'whatever',
 'when/then',
 'whenever',
 'without'}

In [55]:
output_dict = {"all": {"attack": conn_attack_all, "support": conn_support_all, "both": conn_attack_support_all, "all": list(freq_dict.keys())},
               "20": {"attack": conn_attack_20, "support": conn_support_20},
               "40": {"attack": conn_attack_40, "support": conn_support_40},
               "80": {"attack": conn_attack_80, "support": conn_support_80}
              }

In [56]:
output_dict

{'all': {'attack': ['although',
   'though',
   'but',
   'nevertheless',
   'if',
   'previously',
   'when',
   'and',
   'then',
   'while',
   'however',
   'still',
   'or',
   'instead',
   'as',
   'nonetheless',
   'meanwhile',
   'yet',
   'rather',
   'besides',
   'nor',
   'whereas',
   'earlier',
   'conversely'],
  'support': ['but',
   'because',
   'if',
   'when',
   'and',
   'so',
   'then',
   'as',
   'since',
   'therefore',
   'thus',
   'thereby',
   'consequently',
   'hence',
   'accordingly',
   'for',
   'with',
   'given'],
  'both': ['but', 'if', 'when', 'and', 'then', 'as'],
  'all': ['once',
   'although',
   'though',
   'but',
   'because',
   'nevertheless',
   'before',
   'until',
   'if',
   'previously',
   'when',
   'and',
   'so',
   'then',
   'while',
   'however',
   'also',
   'after',
   'separately',
   'still',
   'or',
   'moreover',
   'instead',
   'as',
   'nonetheless',
   'unless',
   'meanwhile',
   'yet',
   'since',
   'rather',

In [57]:
df_attack = pd.DataFrame(filter_connectives(freq_dict, "attacking_perc", 0.01, add_perc=True, add_senses=True), columns=["connective", "%", "senses"]).sort_values("%", ascending=False)
df_attack

,connective,%,senses
23,conversely,100.00,2
21,whereas,100.00,5
2,but,73.28,3308
10,however,73.20,485
9,while,52.50,781
17,yet,52.48,101
11,still,50.53,190
0,although,47.87,328
1,though,47.50,320
3,nevertheless,43.18,44


In [58]:
df_support = pd.DataFrame(filter_connectives(freq_dict, "supportive_perc", 0.01, add_perc=True, add_senses=True), columns=["connective", "%", "senses"]).sort_values("%", ascending=False)
df_support


,connective,%,senses
9,therefore,100.00,26
11,thereby,100.00,12
15,for,100.00,3
14,accordingly,100.00,5
5,so,100.00,263
13,hence,100.00,4
12,consequently,100.00,10
10,thus,100.00,112
1,because,99.53,858
16,with,60.00,5


In [59]:
pd.concat([df_attack, df_support], ignore_index=True, axis=1)

,0,1,2,3,4,5
23,conversely,100.00,2,NaN,NaN,NaN
21,whereas,100.00,5,NaN,NaN,NaN
2,but,73.28,3308,if,0.08,1223.0
10,however,73.20,485,thus,100.00,112.0
9,while,52.50,781,therefore,100.00,26.0
17,yet,52.48,101,given,33.33,3.0
11,still,50.53,190,thereby,100.00,12.0
0,although,47.87,328,but,0.03,3308.0
1,though,47.50,320,because,99.53,858.0
3,nevertheless,43.18,44,when,2.02,989.0


In [60]:
json_content = json.dumps(output_dict)
with open("connectives_dict.json", "w") as f:
    f.write(json_content)

In [22]:
rel_list = list()
freq_sense_dict = dict()

for entry in entries:
    i = 0
    word = entry.get("word")
    if not " " in word:
        ptb_relations = entry.findall(".//pdtb2_relation")
        for rel in ptb_relations:
            sense = rel.get("sense")
            freq = rel.get("pdtb_freq")
            rel_list.append(sense)
            if freq:
                if sense not in freq_sense_dict:
                    freq_sense_dict[sense] = int(freq)
                else:
                    freq_sense_dict[sense] += int(freq)
print(len(freq_sense_dict))
dict(sorted(freq_sense_dict.items(), key=lambda item: item[1], reverse=True))

40


{'Expansion.Conjunction': 4998,
 'Comparison.Contrast': 2319,
 'Temporal.Synchrony': 1389,
 'Contingency.Cause.Reason': 1187,
 'Comparison.Contrast.Juxtaposition': 1099,
 'Temporal.Asynchronous.Succession': 1053,
 'Temporal.Asynchronous.Precedence': 928,
 'Comparison.Concession.Contra-expectation': 793,
 'Contingency.Condition.Hypothetical': 719,
 'Contingency.Cause.Result': 502,
 'Comparison': 390,
 'Comparison.Concession.Expectation': 384,
 'Comparison.Contrast.Opposition': 330,
 'Contingency.Condition.General': 315,
 'Expansion.List': 240,
 'Expansion.Alternative.Disjunctive': 138,
 'Contingency.Condition.Unreal present': 123,
 'Expansion.Alternative.Chosen alternative': 113,
 'Contingency.Condition.Factual present': 79,
 'Expansion.Restatement.Specification': 57,
 'Contingency.Condition.Unreal past': 53,
 'Expansion.Alternative.Conjunctive': 46,
 'Contingency.Pragmatic condition.Implicit assertion': 45,
 'Expansion.Alternative': 45,
 'Contingency.Pragmatic condition.Relevance': 21,

In [10]:
connectives = [conn.split("\t")[0] for conn in connective_string.split("\n")]
connectives

['because',
 'therefore',
 'still',
 'since',
 'as',
 'consequently',
 'hence',
 'thus',
 'so',
 'accordingly',
 'moreover',
 'furthermore',
 'also',
 'likewise',
 'additionally',
 'similarly',
 'and',
 'further',
 'unless',
 'although',
 'though',
 'but',
 'whereas',
 'while',
 'however',
 'nevertheless',
 'yet',
 'nonetheless',
 'conversely',
 'despite']

In [26]:
supportive = ["because", "accordingly", "since", "as", "consequently", "hence", "thus", "so", "therefore"]
rather_supportive = ["moreover", "furthermore", "also", "likewise", "additionally", "similarly", "and", "further"]
attacking = (set(supportive).union(set(rather_supportive))).symmetric_difference(set(connectives))
print(attacking)

{'though', 'unless', 'while', 'but', 'still', 'conversely', 'however', 'despite', 'although', 'whereas', 'nevertheless', 'nonetheless', 'yet'}


In [43]:
relevant_entries = dict()
i = 0
for entry in entries:
    word = entry.get("word")
    cat = entry.find(".//cat")
    if not " " in word:
        i += 1
        ptb_relations = entry.findall(".//pdtb2_relation")
        ptb_senses = [rel.get("sense") for rel in ptb_relations]
        if word in supportive:
            # print(word, ptb_senses)
            # ptb_main_senses = [rel for rel in ptb_senses if not "." in rel]
            relevant_entries[word] = set(ptb_senses)
print("one-token connectives:", i, "multi-token connectives:", len(entries)-i)
relevant_entries

one-token connectives: 79 multi-token connectives: 63


{'because': {'Contingency.Cause.Reason',
  'Contingency.Pragmatic cause.Justification',
  'Contingency.Pragmatic condition.Implicit assertion'},
 'so': {'Contingency.Cause.Reason', 'Contingency.Cause.Result'},
 'as': {'Comparison',
  'Comparison.Concession.Contra-expectation',
  'Comparison.Concession.Expectation',
  'Comparison.Contrast.Opposition',
  'Contingency.Cause.Reason',
  'Contingency.Pragmatic cause.Justification',
  'Expansion',
  'Expansion.Conjunction',
  'Temporal.Asynchronous.Succession',
  'Temporal.Synchrony'},
 'since': {'Contingency.Cause.Reason',
  'Temporal.Asynchronous.Precedence',
  'Temporal.Asynchronous.Succession',
  'Temporal.Synchrony'},
 'therefore': {'Contingency.Cause.Result'},
 'thus': {'Contingency.Cause.Result'},
 'consequently': {'Contingency.Cause.Result'},
 'hence': {'Contingency.Cause.Result'},
 'accordingly': {'Contingency.Cause.Result'}}

- all support either contingency.cause.reason or contingency.cause.result
- all rather supportive Expansion.Conjunction
- all attack are Comparison.Contrast

exceptions: despite, unless (all attack)

In [75]:
# supportive

relevant_entries = dict()
for entry in entries:
    word = entry.get("word")
    cat = entry.find(".//cat")
    if not " " in word:
        ptb_relations = entry.findall(".//pdtb2_relation")
        for rel in ptb_relations:
            # ptb_senses = [rel.get("sense") for rel in ptb_relations]
            # ptb_freq = [rel.get("pdtb_freq") for rel in ptb_relations]
            freq = rel.get("pdtb_freq")
            sense = rel.get("sense")
            #for sense, freq in zip(ptb_senses, ptb_freq):
            if sense in ["Contingency.Cause.Result", "Contingency.Cause.Reason", "Contigency.Pragmatic cause.Justification", "Contigency.Pragmatic Cause.Justification"]:  # "Expansion.Conjunction":  # "Comparison.Contrast":
                if freq:
                    if word in supportive:
                        print(word, freq, freq_dict[word], round(((int(freq)/freq_dict[word])*100),2), "supportive", sep="\t")
                    else:
                        print(word, freq, freq_dict[word], round(((int(freq)/freq_dict[word])*100),2), sep="\t")   # , cat.text)  # , ptb_senses)
                # break
            # ptb_main_senses = [rel for rel in ptb_senses if not "." in rel]
            # relevant_entries[word] = set(ptb_senses)
relevant_entries

but	1	3308	0.03
because	73	858	8.51	supportive
because	14	858	1.63	supportive
because	767	858	89.39	supportive
if	1	1223	0.08
when	1	989	0.1
when	19	989	1.92
and	65	3000	2.17
so	13	263	4.94	supportive
so	49	263	18.63	supportive
so	61	263	23.19	supportive
so	139	263	52.85	supportive
so	1	263	0.38	supportive
then	4	340	1.18
then	1	340	0.29
as	3	743	0.4	supportive
as	4	743	0.54	supportive
as	205	743	27.59	supportive
since	6	184	3.26	supportive
since	90	184	48.91	supportive
therefore	26	26	100.0	supportive
thus	103	112	91.96	supportive
thus	9	112	8.04	supportive
thereby	12	12	100.0
consequently	10	10	100.0	supportive
hence	4	4	100.0	supportive
accordingly	5	5	100.0	supportive
for	3	3	100.0


{}

In [77]:
# rather supportive

relevant_entries = dict()
for entry in entries:
    word = entry.get("word")
    cat = entry.find(".//cat")
    if not " " in word:
        ptb_relations = entry.findall(".//pdtb2_relation")
        for rel in ptb_relations:
            # ptb_senses = [rel.get("sense") for rel in ptb_relations]
            # ptb_freq = [rel.get("pdtb_freq") for rel in ptb_relations]
            freq = rel.get("pdtb_freq")
            sense = rel.get("sense")
            #for sense, freq in zip(ptb_senses, ptb_freq):
            if sense.startswith("Expansion.Conjunction"):  # "Expansion.Conjunction":  # "Comparison.Contrast":
                if freq:
                    if word in rather_supportive:
                        print(word, freq, freq_dict[word], round(((int(freq)/freq_dict[word])*100),2), "rather supportive", sep="\t")
                    else:
                        print(word, freq, freq_dict[word], round(((int(freq)/freq_dict[word])*100),2), sep="\t")   # , cat.text)  # , ptb_senses)
                # break
            # ptb_main_senses = [rel for rel in ptb_senses if not "." in rel]
            # relevant_entries[word] = set(ptb_senses)
relevant_entries

but	82	3308	2.48
and	2689	3000	89.63	rather supportive
and	1	3000	0.03	rather supportive
and	1	3000	0.03	rather supportive
then	8	340	2.35
then	3	340	0.88
while	54	781	6.91
however	2	485	0.41
also	1714	1746	98.17	rather supportive
also	20	1746	1.15	rather supportive
separately	72	74	97.3
or	5	98	5.1
moreover	100	101	99.01	rather supportive
as	1	743	0.13
nonetheless	1	27	3.7
meanwhile	70	193	36.27
yet	2	101	1.98
yet	1	101	0.99
indeed	54	104	51.92
later	1	91	1.1
ultimately	2	18	11.11
further	8	9	88.89	rather supportive
further	1	9	11.11	rather supportive
next	1	7	14.29
similarly	18	18	100.0	rather supportive
besides	7	19	36.84
besides	10	19	52.63
nor	25	31	80.65
overall	3	12	25.0
finally	12	32	37.5
additionally	7	7	100.0	rather supportive
meantime	1	15	6.67
meantime	1	15	6.67
likewise	8	8	100.0	rather supportive
furthermore	11	11	100.0	rather supportive
specifically	1	10	10.0
plus	1	1	100.0


{}

In [78]:
# supportive

relevant_entries = dict()
for entry in entries:
    word = entry.get("word")
    cat = entry.find(".//cat")
    if not " " in word:
        ptb_relations = entry.findall(".//pdtb2_relation")
        for rel in ptb_relations:
            # ptb_senses = [rel.get("sense") for rel in ptb_relations]
            # ptb_freq = [rel.get("pdtb_freq") for rel in ptb_relations]
            freq = rel.get("pdtb_freq")
            sense = rel.get("sense")
            #for sense, freq in zip(ptb_senses, ptb_freq):
            if sense.startswith("Comparison.Contrast") or sense == "Comparison.Pragmatic contrast":  # "Expansion.Conjunction":  # "Comparison.Contrast":
                if freq:
                    if word in attacking:
                        print(word, freq, freq_dict[word], round(((int(freq)/freq_dict[word])*100),2), "attack", sep="\t")
                    else:
                        print(word, freq, freq_dict[word], round(((int(freq)/freq_dict[word])*100),2), sep="\t")   # , cat.text)  # , ptb_senses)
                # break
            # ptb_main_senses = [rel for rel in ptb_senses if not "." in rel]
            # relevant_entries[word] = set(ptb_senses)
relevant_entries

although	34	328	10.37	attack
although	9	328	2.74	attack
although	114	328	34.76	attack
though	5	320	1.56	attack
though	1	320	0.31	attack
though	15	320	4.69	attack
though	1	320	0.31	attack
though	2	320	0.62	attack
though	1	320	0.31	attack
though	14	320	4.38	attack
though	1	320	0.31	attack
though	33	320	10.31	attack
though	53	320	16.56	attack
though	26	320	8.12	attack
but	638	3308	19.29	attack
but	175	3308	5.29	attack
but	16	3308	0.48	attack
but	1609	3308	48.64	attack
but	1	3308	0.03	attack
but	1	3308	0.03	attack
nevertheless	7	44	15.91	attack
nevertheless	1	44	2.27	attack
nevertheless	11	44	25.0	attack
if	1	1223	0.08
if	4	1223	0.33
if	1	1223	0.08
previously	2	49	4.08
when	1	989	0.1
when	1	989	0.1
and	13	3000	0.43
and	3	3000	0.1
and	5	3000	0.17
then	1	340	0.29
while	1	781	0.13	attack
while	2	781	0.26	attack
while	195	781	24.97	attack
while	84	781	10.76	attack
while	128	781	16.39	attack
however	89	485	18.35	attack
however	32	485	6.6	attack
however	1	485	0.21	attack
however	234	485	48.25	at

{}

- attacking: 
    - missing in Comparison.Contrast: if, when, then, or, instead, meanwhile, rather, nor, 
    - in other categories: and, 
- rather supportive:
    - in other categories: but, while, however, as, nonetheless, yet, 
    - missing: then, separately, or, meanwhile, indeed, later, ultimately, next, besides, nor, overall, finally, meantime, specifically, plis, without, with, anyway
- supportive:
    - in other categories: but, then, 
    - missing: if, when, and, thereby, for, with, given

if we would pick all conenctives with a higher usage than 85% in the selected category
- supportive:
    - as (28%) and since (52%) would be kicked out (as to often used as temporal usage)
- rather supportive:
    - nor (80%, freq: 25) and separately (97%, freq: 74) and plus (100%, but freq: 1) should be added
- attack:
    - many connectives would be kicked out, the percentage would be too low or also occur too less
    - altough, though, but, nevertheless, conversely, while, however, still, nonetheless, whereas
    - rather should be added
    - despite and unless would be kicked as different categories?

In [15]:
# supportive

relevant_entries = dict()
for entry in entries:
    word = entry.get("word")
    cat = entry.find(".//cat")
    #if not " " in word:
    ptb_relations = entry.findall(".//pdtb2_relation")
    for rel in ptb_relations:
        # ptb_senses = [rel.get("sense") for rel in ptb_relations]
        # ptb_freq = [rel.get("pdtb_freq") for rel in ptb_relations]
        freq = rel.get("pdtb_freq")
        sense = rel.get("sense")
        #for sense, freq in zip(ptb_senses, ptb_freq):
        print(word, sense)
            # if sense.startswith("Comparison.Contrast"):  # "Expansion.Conjunction":  # "Comparison.Contrast":
            #     if freq:
                    # if word in attacking:
                    #     print(word, freq, freq_dict[word], round(((int(freq)/freq_dict[word])*100),2), "attack", sep="\t")
                    # else:
                    #     print(word, freq, freq_dict[word], round(((int(freq)/freq_dict[word])*100),2), sep="\t")   # , cat.text)  # , ptb_senses)
                # break
            # ptb_main_senses = [rel for rel in ptb_senses if not "." in rel]
            # relevant_entries[word] = set(ptb_senses)
relevant_entries

once Temporal.Asynchronous.Succession
once Temporal.Synchrony
once Contingency.Condition.General
once Temporal.Asynchronous.Succession
once Contingency.Condition.Hypothetical
once Contingency.Condition.General
once Temporal.Asynchronous.Succession
although Comparison
although Expansion.Exception
although Comparison.Concession
although Comparison.Contrast.Juxtaposition
although Comparison.Contrast.Opposition
although Comparison.Concession.Contra-expectation
although Comparison.Contrast
although Comparison.Concession.Expectation
though Comparison
though Comparison.Concession
though Comparison.Contrast.Juxtaposition
though Comparison.Contrast.Opposition
though Comparison.Concession.Contra-expectation
though Comparison.Concession.Expectation
though Comparison.Contrast
though Comparison.Concession.Expectation
though Comparison.Contrast.Juxtaposition
though Comparison.Contrast
though Comparison.Contrast.Opposition
though Comparison
though Comparison.Concession
though Comparison.Contrast.Juxt

{}